# BDPBot

---

A bot based on BERT using transfer learning
The dataset to train and test the model will be available of interact in english, but no in spanish

In [ ]:
# load bert tokenizer and Classification from transformers
from transformers import BertTokenizer, BertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

Now we need to tokenize the texts that are going to be used for train and test the model in this case, i am going to use TriviaQA from http://nlp.cs.washington.edu/triviaqa/

In [ ]:
# Download dataset if it doesn't exist

import requests
import tarfile
import os
url = "http://nlp.cs.washington.edu/triviaqa/data/triviaqa-unfiltered.tar.gz"
target_path = 'triviaqa-unfiltered.tar.gz'

if target_path not in os.listdir():
    print("Downloading file")
    # Dowload the file showing the progress bar
    
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        print("Saving file")
        with open(target_path, 'wb') as f:
            f.write(response.raw.read())

    # Unzip the file
    with tarfile.open(target_path, "r:gz") as tar:
        tar.extractall(path=".")
else:
    print("File already exists")

In [ ]:
# create a dataframe from the json file with the variables that i need 

import pandas as pd
import json

if 'triviaqa-unfiltered.csv' not in os.listdir():
    # Load Json file
    with open('./triviaqa-unfiltered/unfiltered-web-train.json', 'r') as file:
        data = json.load(file)
    if isinstance(data, dict):
        data_to_df = []
        data_json = data["Data"]
        for item in data_json:
            question = item.get('Question', '')  # Usar get para evitar errores si la clave no existe
            answer = item.get('Answer', {}).get('Value', '')  # Anidando get para acceder a 'Value'
            search_results = item.get('SearchResults', [])
            context = ""
            for searched in search_results:
                context += searched.get('Description', '') + " "
            data_to_df.append({
                'Question': question,
                'Answer': answer,
                'Context': context
            })

        # Create a DataFrame from the list of dictionaries
        df = pd.DataFrame(data_to_df)
        # Save the DataFrame to a csv file
        df.to_csv('triviaqa-unfiltered.csv', index=False)
        # Visualizar las primeras filas del DataFrame
        print(df.head())
    else:
        print("Json has not the dxpected format")
else:
    df = pd.read_csv('triviaqa-unfiltered.csv')
    print(df.head())

In [ ]:
if 'test-triviaqa-unfiltered.csv' not in os.listdir():
    # Load Json file
    with open('./triviaqa-unfiltered/unfiltered-web-dev.json', 'r') as file:
        data = json.load(file)
    if isinstance(data, dict):
        data_to_df = []
        data_json = data["Data"]
        for item in data_json:
            question = item.get('Question', '')  # Usar get para evitar errores si la clave no existe
            answer = item.get('Answer', {}).get('Value', '')  # Anidando get para acceder a 'Value'
            search_results = item.get('SearchResults', [])
            context = ""
            for searched in search_results:
                context += searched.get('Description', '') + " "
            data_to_df.append({
                'Question': question,
                'Answer': answer,
                'Context': context
            })

        # Create a DataFrame from the list of dictionaries
        test = pd.DataFrame(data_to_df)
        # Save the DataFrame to a csv file
        test.to_csv('test-triviaqa-unfiltered.csv', index=False)
        # Visualizar las primeras filas del DataFrame
        print(df.head())
    else:
        print("Json has not the dxpected format")
else:
    test = pd.read_csv('test-triviaqa-unfiltered.csv')
    print(test.head())